In [68]:
import numpy as np
import gym
from gym_ianna.envs.ianna_env import IANNAEnv
import tensorflow as tf


In [69]:
env_d = 'IANNA-v0'
#env_d = 'CartPole-v0'

if env_d == 'IANNA-v0':
    OP_NUMBER = 15
    STATE_INPUT_SIZE = 32
    MAX_STEPS = 20

if env_d == 'CartPole-v0':
    OP_NUMBER = 2
    STATE_INPUT_SIZE = 4
    MAX_STEPS = 200

HIDDEN_SIZE = 30


In [70]:
#Initializing 
tf.reset_default_graph()

W1 = tf.get_variable(shape=[HIDDEN_SIZE,STATE_INPUT_SIZE],name='w1',
                      initializer=tf.contrib.layers.xavier_initializer())
W2 = tf.get_variable(shape=[HIDDEN_SIZE,HIDDEN_SIZE],name='w2',
                      initializer=tf.contrib.layers.xavier_initializer())
W3 = tf.get_variable(shape=[OP_NUMBER,HIDDEN_SIZE],name='w3',
                      initializer=tf.contrib.layers.xavier_initializer())

b1 = tf.get_variable(shape=[HIDDEN_SIZE,1],name='b1',
                      initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.get_variable(shape=[HIDDEN_SIZE,1],name='b2',
                      initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.get_variable(shape=[OP_NUMBER,1],name='b3',
                      initializer=tf.contrib.layers.xavier_initializer())

#Layers:
x = tf.placeholder(tf.float32, shape=[STATE_INPUT_SIZE,None],name='x')
h1 = tf.tanh(tf.matmul(W1,x) + b1)
h2 = tf.tanh(tf.matmul(W2,h1) + b2)
y = tf.nn.softmax(tf.matmul(W3,h2) + b3,dim=0)


In [71]:
saver = tf.train.Saver()
sess = tf.InteractiveSession()



In [72]:
#Loss function:
curr_reward = tf.placeholder(shape=[None],dtype=tf.float32)
actions_array = tf.placeholder(shape=[None],dtype=tf.int32)
pai_array = tf.gather(y,actions_array)
L = -tf.reduce_mean(tf.log(pai_array)*curr_reward)
gradient_holders = []
gradients = tf.gradients(L,tf.trainable_variables())


/home/ihadanny/anaconda2/envs/py3k/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [73]:
tvars = tf.trainable_variables()
#Initialize gradient lists for each trainable variable:
for idx,var in enumerate(tvars):
    placeholder = tf.placeholder(tf.float32,name=str(idx)+'_holder')
    gradient_holders.append(placeholder)

In [74]:
#ADAM Optimizer hyper-parameters:
LEARNING_RATE = 0.01
B1 = 0.8
B2= 0.999
EPSILON=1e-6

#Update mechanism:
adam = tf.train.AdamOptimizer(learning_rate=LEARNING_RATE,beta1=B1,beta2=B2,epsilon=EPSILON)
update_batch = adam.apply_gradients(zip(gradient_holders,tvars))


In [75]:
# grad buffer is initialized to all zeros. 
# It's used to accumulate the gradients and is a regular variable, NOT a tf variable
init = tf.global_variables_initializer()
sess.run(init)
#saver.restore(sess, "../models/ianna-nn-supervised")

grad_buffer = sess.run(tf.trainable_variables())

def reset_grad_buffer():
    for ix,grad in enumerate(grad_buffer):
        grad_buffer[ix] = grad * 0


In [76]:
def get_action(sess,observation):
    """
    Given an observation, return action sampled according to the probabilities of the NN output: 
    """
    a_dist = sess.run(y,feed_dict={x:np.reshape(observation,(STATE_INPUT_SIZE, 1))})
    a = np.random.choice(range(OP_NUMBER),p=a_dist.reshape((OP_NUMBER)))        
    return a


In [77]:
def train(sess,cur_states_array,cur_actions_array,cur_curr_reward):
    """
    NN training procedure: Given arrays of states(observations),
    actions and rewards it computes the derivatives of the loss function
    then add the derivation values to the buffer, 
    """

    G = sess.run(gradients,feed_dict={x:cur_states_array,actions_array:cur_actions_array,curr_reward:cur_curr_reward})
    for idx,grad in enumerate(G):
        grad_buffer[idx] += grad


In [78]:
def update(sess):
    """
    NN update procedure: apply the gradients to the NN variables
    """
    feed_dict = dict(zip(gradient_holders, grad_buffer))
    _ = sess.run(update_batch, feed_dict=feed_dict)


In [79]:
#        IANNA actions would be:
#        0) action_type:            back[0], filter[1], group[2]
#        1) col_id:                 [0..num_of_columns-1]
#        2) filter_operator:        LT[0], GT[1] if the selected column was numeric (maybe change semantics if column is STR?)
#        3) filter_decile:          [0..9] the filter operand  
#        4) aggregation column_id:  [0..num_of_columns - 1] (what do we do if the selected col is also grouped_by?)
#        5) aggregation type:       MEAN[0], COUNT[1], SUM[2], MIN[3], MAX[4]

def build_ianna_action_from_grouped_by_field(grouped_by_field):
    action = [2, grouped_by_field, 0, 0, 0, 0]
    return action

In [80]:
gamma = 0.99

def discount_rewards(arr):
    """
    Helper function for computing discounted rewards,
    then the delayed rewards are normalized by the mean and std as requested.
    """
    discounts = np.zeros_like(arr)
    reward = 0
    for i in reversed(range(arr.size)):
        reward=gamma*(arr[i]+reward)
        discounts[i] = reward
    mean = np.mean(discounts,keepdims=True)    
    discounts = discounts - mean
    discounts = discounts/ np.std(discounts)
    return discounts


In [81]:
TOTAL_EPISODES = 300
BATCH_NUMBER = 10
DISPLAY_FREQ = 50

env = gym.make(env_d)
total_episodes=TOTAL_EPISODES
batch_number = BATCH_NUMBER



[2017-08-19 17:29:27,031] Making new env: IANNA-v0


reading input /home/ihadanny/src/IANNA/gym_ianna/envs/../../data/1.tsv
observation space from [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] to [8648 8648   10    2    2    3 8148    0 3692  180   10 8648    1  118  157
  208  206    1    1    1    1    1    1    1    1    1    1    1    1    1
    1    1] shape (32,)
action space MultiDiscrete6


In [82]:
episode_number = 0

rewards = []
steps=[]
max_reward=0

reset_grad_buffer()

In [83]:
while episode_number <= total_episodes:
    for ep in range(batch_number):
        obsrv = env.reset()
        ep_history=[]
        step_num=0
        total_reward=0
        done=False

        while not done and step_num < MAX_STEPS:
            #Perform the game "step:"
            step_num+=1
            action = get_action(sess,obsrv)
            if env_d == 'IANNA-v0':
                complex_action = build_ianna_action_from_grouped_by_field(action)
                obsrv1, reward, done, info = env.step(complex_action)
            else:
                obsrv1, reward, done, info = env.step(action)

            total_reward+=reward
            ep_history.append((np.array(obsrv),action,reward))
            obsrv=obsrv1

        episode_number+=1
        ep_history= np.array(ep_history)               
        ep_history[:,2] = discount_rewards(ep_history[:,2])

        """
        perform the training step, 
        feeding the network with the ep_history that contains
        the states,actions, and discounted rewards
        """
        ep_states_array = np.vstack(ep_history[:,0]).T
        ep_actions_array = ep_history[:,1].T
        ep_curr_reward = ep_history[:,2].T 
        L=train(sess, ep_states_array, ep_actions_array, ep_curr_reward)

        #update the rewards/steps counter, storing the data for all episodes
        rewards.append(total_reward)
        steps.append(step_num)    
        
        if episode_number%DISPLAY_FREQ==0:
            print("Total episodes: %d"%episode_number)
            print("Average steps: %f"%np.mean(steps[-batch_number:]))
            print("Average reward: %f"%np.mean(rewards[-batch_number:]))
    update(sess)
    reset_grad_buffer()
    if np.mean(rewards[-batch_number:])>max_reward:
        max_reward=np.mean(rewards[-batch_number:])
        print("\t\t\tCurr Max mean reward:",max_reward)
  
        

			Curr Max mean reward: 6.9
Total episodes: 50
Average steps: 10.000000
Average reward: 4.700000
Total episodes: 100
Average steps: 10.000000
Average reward: 5.100000
Total episodes: 150
Average steps: 10.000000
Average reward: 3.200000
Total episodes: 200
Average steps: 10.000000
Average reward: 2.800000
Total episodes: 250
Average steps: 10.000000
Average reward: 2.100000
Total episodes: 300
Average steps: 10.000000
Average reward: 2.300000


In [84]:
sess.close()